In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.models import load_model
import cv2
import os
from sklearn.model_selection import train_test_split

# First, load the model with compile=False to avoid optimizer warnings
model = load_model(r'isFinal.h5', compile=False)

# Recompile the model with the necessary metrics and optimizer
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

def load_images_from_folder(folder, label, max_images=3500):
    images = []
    labels = []
    all_files = os.listdir(folder)

    # Shuffle and select a subset of files
    np.random.shuffle(all_files)
    max_images = max_images or len(all_files) // 2

    for filename in all_files[:max_images]:
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            resized_img = cv2.resize(img, (64, 64))  # Resize to 128x128
            images.append(resized_img)
            labels.append(label)

    # Convert to NumPy arrays and add channel dimension
    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Load the images
real_images, real_labels = load_images_from_folder(r'D:\SIH project\dataset2\Dataset\Train\Real', 0)
fake_images, fake_labels = load_images_from_folder(r'D:\SIH project\dataset2\Dataset\Train\Fake', 1)

# Combine real and fake data
x = np.concatenate((real_images, fake_images), axis=0)
y = np.concatenate((real_labels, fake_labels), axis=0)
    
# Split the data
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

# Normalize the pixel values
xtrain = xtrain / 255.0
xtest = xtest / 255.0

print(f"Training data shape: {xtrain.shape}, Labels shape: {ytrain.shape}")

Training data shape: (5600, 64, 64, 3), Labels shape: (5600,)


In [15]:
model.evaluate(x,y)

219/219 ━━━━━━━━━━━━━━━━━━━━ 87s 395ms/step - accuracy: 0.9456 - loss: 0.1262


[0.13000768423080444, 0.9494285583496094]

In [4]:
test_real_images, test_real_labels = load_images_from_folder(r'D:\SIH project\dataset2\Dataset\Test\Real', 0)
test_fake_images, test_fake_labels = load_images_from_folder(r'D:\SIH project\dataset2\Dataset\Test\Fake', 1)

# Combine real and fake data
testData = np.concatenate((test_real_images, test_fake_images), axis=0)
testLabels = np.concatenate((test_real_labels, test_fake_labels), axis=0)

In [5]:
model.evaluate(testData,testLabels)

219/219 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.9277 - loss: 27.1221


[51.5689582824707, 0.8830000162124634]

In [16]:
val_real_images, val_real_labels = load_images_from_folder(r'D:\SIH project\dataset2\Dataset\Validation\Real', 0,max_images=None)
val_fake_images, val_fake_labels = load_images_from_folder(r'D:\SIH project\dataset2\Dataset\Validation\Fake', 1,max_images=None)

# Combine real and fake data
valData = np.concatenate((val_real_images, val_fake_images), axis=0)
valLabels = np.concatenate((val_real_labels, val_fake_labels), axis=0)

In [17]:
model.evaluate(valData,valLabels)

617/617 ━━━━━━━━━━━━━━━━━━━━ 255s 413ms/step - accuracy: 0.9252 - loss: 0.1793


[0.22787314653396606, 0.9121899008750916]

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = r'D:\SIH project\20210119_210738.jpg'
img = image.load_img(img_path, target_size=(128, 128))  # Resize to 224x224 if that's what your model expects
img_array = image.img_to_array(img)  # Convert to array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize the image

# Load your pre-trained deep fake analysis model
#model = tf.keras.models.load_model('path_to_your_model.h5')

# Pass the image through the model
predictions = model.predict(img_array)

# Interpret the results
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
[[0.54456055]]
